In [1]:
!pip install pymongo
from pymongo import MongoClient

In [2]:
host="rhea.isegi.unl.pt"
port="28011"
user="GROUP_11"
password="MTM0MzQ5MzI3NjY4ODM1MzQ2ODc2OTMzMjgyNjQ3NjYyNDcyNjUw"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [3]:
db = client.contracts
eu = db.eu

In [4]:
bot_year = 2008
top_year = 2020
country_list = ['NO', 'HR', 'HU', 'CH', 'CZ', 'RO', 'LV', 'GR', 'UK', 'SI', 'LT', 'ES', 'FR', 'IE', 'SE', 'NL',
                'PT', 'PL', 'DK', 'MK', 'DE', 'IT', 'BG', 'CY', 'AT', 'LU', 'BE', 'FI', 'EE', 'SK', 'MT', 'LI',
                'IS']

12. Returns the average 'Value_Euro' for each country, return the highest 5 countries

In [5]:
project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'NUMBER_OFFERS':True,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]}
    }
}

group = {
    '$group':{
        '_id':{'YEAR':'$YEAR','COUNTRY':'$ISO_COUNTRY_CODE','CPV_DIVISION':'$CPV_DIVISION'},
        'AVERAGE':{'$avg':'$VALUE_EURO'},
        'COUNT':{'$sum':1}
    }
}

limit = {
    '$limit':5
}

13. Returns the average 'VALUE_EURO' for each country, return the lowest 5 countries
 

In [ ]:
project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'NUMBER_OFFERS':True,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]}
    }
}

group = {
    '$group':{
        '_id':{'YEAR':'$YEAR','COUNTRY':'$ISO_COUNTRY_CODE','CPV_DIVISION':'$CPV_DIVISION'},
        'AVERAGE':{'$avg':'$VALUE_EURO'},
        'COUNT':{'$sum':1}
    }
}

# return the lowest five
limit = {
    '$limit':5
}

14. For each country get the sum of the respective contracts 'VALUE_EURO' which recieved european funds ('B_EU_FUNDS') 

16. Returns the average 'VALUE_EURO' for company ('CAE_NAME') return the highest 5 companies

In [ ]:
project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'NUMBER_OFFERS':True,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]}
    }
}

group = {
    '$group':{
        '_id':{'YEAR':'$YEAR','COMPANY':'$CAE_NAME','CPV_DIVISION':'$CPV_DIVISION'},
        'AVERAGE':{'$avg':'$VALUE_EURO'},
        'COUNT':{'$sum':1}
    }
}


17. Returns the average 'VALUE_EURO' for company ('CAE_NAME') return the lowest 5 companies

In [ ]:
project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'NUMBER_OFFERS':True,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]}
    }
}

group = {
    '$group':{
        '_id':{'YEAR':'$YEAR','COMPANY':'$CAE_NAME'','CPV_DIVISION':'$CPV_DIVISION'},
        'AVERAGE':{'$avg':'$VALUE_EURO'},
        'COUNT':{'$sum':1}
    }
}
